# Sampling Raster Data using Points or Polygons 

## Task
Given a raster grid of daily maximum temperature in the continental US:
   - extract the temperature at a point layer of all urban areas 
   - calculate the average temperature for a polygon layer of each county in the US.
   
   

In [31]:
# Create relative path

import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio  
from rasterstats import zonal_stats

data_pkg_path = 'data/gpkg'
filename = 'us_country_data.gpkg'
path = os.path.join(data_pkg_path, filename)
print(path)

data/gpkg/us_country_data.gpkg


In [32]:
# Read Files

urban_areas = gpd.read_file(path, layer='2018_Gaz_ua_national')
census_tracts = gpd.read_file(path, layer = 'tl_2018_us_county')
print(urban_areas.info(), census_tracts.info())
urban_areas.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3601 entries, 0 to 3600
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       3601 non-null   int64   
 1   NAME        3601 non-null   object  
 2   UATYPE      3601 non-null   object  
 3   ALAND       3601 non-null   float64 
 4   AWATER      3601 non-null   int64   
 5   ALAND_SQMI  3601 non-null   float64 
 6   AWATER_SQM  3601 non-null   float64 
 7   INTPTLAT    3601 non-null   float64 
 8   INTPTLONG   3601 non-null   float64 
 9   geometry    3601 non-null   geometry
dtypes: float64(5), geometry(1), int64(2), object(2)
memory usage: 281.5+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   3233 non-null   object  
 1   COUNTYFP  3233 non-null   object  
 2   COUNTYNS  3233 non-null 

,GEOID,NAME,UATYPE,ALAND,AWATER,ALAND_SQMI,AWATER_SQM,INTPTLAT,INTPTLONG,geometry
0,37,"Abbeville, LA Urban Cluster",C,29189594.0,298416,11.270,0.115,29.967156,-92.095966,POINT (-92.09597 29.96716)
1,64,"Abbeville, SC Urban Cluster",C,11271121.0,19786,4.352,0.008,34.179273,-82.379776,POINT (-82.37978 34.17927)
2,91,"Abbotsford, WI Urban Cluster",C,5394605.0,13221,2.083,0.005,44.948612,-90.315875,POINT (-90.31588 44.94861)
3,118,"Aberdeen, MS Urban Cluster",C,7416339.0,52820,2.863,0.020,33.824742,-88.554591,POINT (-88.55459 33.82474)
4,145,"Aberdeen, SD Urban Cluster",C,33035011.0,120864,12.755,0.047,45.463186,-98.471033,POINT (-98.47103 45.46319)


In [33]:
# Create relative path for raster data

data_rast_path = 'data/raster'
rast_filename = 'us.tmax_nohads_ll_20220717_float.tif'
rast_path = os.path.join(data_rast_path, rast_filename)
print(rast_path)

data/raster/us.tmax_nohads_ll_20220717_float.tif


In [34]:
# Read raster data and show metadata

us_tmax = rasterio.open(rast_path)
metadata = us_tmax.meta
metadata

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 141,
 'height': 71,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.5, 0.0, -130.25,
        0.0, -0.5, 55.25)}

In [35]:
# Extract coordinates values from the point layer
coord_list = [(x,y) for x,y in zip(urban_areas['geometry'].x , urban_areas['geometry'].y)]

print(type (coord_list))

<class 'list'>


# Sample raster data using points

we use the `rasterio` library (`.sample()`)

In [36]:
urban_areas['tmax'] = [x for x in us_tmax.sample(coord_list)]

# convert tmax col type from list to float so that the field is valid for writing with .to_file
urban_areas['tmax'] = urban_areas['tmax'].astype('float64')

print(urban_areas.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3601 entries, 0 to 3600
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       3601 non-null   int64   
 1   NAME        3601 non-null   object  
 2   UATYPE      3601 non-null   object  
 3   ALAND       3601 non-null   float64 
 4   AWATER      3601 non-null   int64   
 5   ALAND_SQMI  3601 non-null   float64 
 6   AWATER_SQM  3601 non-null   float64 
 7   INTPTLAT    3601 non-null   float64 
 8   INTPTLONG   3601 non-null   float64 
 9   geometry    3601 non-null   geometry
 10  tmax        3601 non-null   float64 
dtypes: float64(6), geometry(1), int64(2), object(2)
memory usage: 309.6+ KB
None


In [37]:
output_dir = 'output'
output_filename = 'us_tmp_country_data.gpkg'
output_path = os.path.join(output_dir, output_filename)

urban_areas.to_file(driver='GPKG', filename=output_path, layer = 'urban_areas_t',  encoding='utf-8')
print('Successfully written output file at {}'.format(output_path))

Successfully written output file at output/us_tmp_country_data.gpkg


# Calculate Zonal Statistics (Mean) 

we use the `rasterstats` library

Use `%%capture` to avoid visualising all the warnings about no data (issue with `rasterstats`?)

In [38]:
%%capture

stats = zonal_stats(census_tracts, rast_path, stats = ['mean'])

print(stats) # Issue: Too many NULL values, why?

In [39]:
# Convert the output dictionary in a pandas dataframe
tempstats_df = pd.DataFrame(stats)

print(tempstats_df)

           mean
0     30.453856
1           NaN
2     35.639346
3     30.471180
4           NaN
...         ...
3228        NaN
3229        NaN
3230  34.299095
3231  38.142677
3232  29.840471

[3233 rows x 1 columns]


In [40]:
# use pandas.concat to append the output to the original geodataframe and save to the new geopakcage

census_tracts_t = pd.concat([census_tracts, tempstats_df], axis=1)
print(testing)

# close the raster just in case
us_tmax.close()

census_tracts_t.to_file(driver='GPKG', filename = output_path, layer = 'census_tracts_t',  encoding='utf-8')
print('Successfully written output file at {}'.format(output_path))

     STATEFP COUNTYFP  COUNTYNS  GEOID       NAME          NAMELSAD LSAD  \
0         31      039  00835841  31039     Cuming     Cuming County   06   
1         53      069  01513275  53069  Wahkiakum  Wahkiakum County   06   
2         35      011  00933054  35011    De Baca    De Baca County   06   
3         31      109  00835876  31109  Lancaster  Lancaster County   06   
4         31      129  00835886  31129   Nuckolls   Nuckolls County   06   
...      ...      ...       ...    ...        ...               ...  ...   
3228      13      123  00351260  13123     Gilmer     Gilmer County   06   
3229      27      135  00659513  27135     Roseau     Roseau County   06   
3230      28      089  00695768  28089    Madison    Madison County   06   
3231      48      227  01383899  48227     Howard     Howard County   06   
3232      54      099  01550056  54099      Wayne      Wayne County   06   

     CLASSFP  MTFCC CSAFP CBSAFP METDIVFP FUNCSTAT       ALAND    AWATER  \
0         H

# Issues

- There are too many Null values 
    - In QGIS, as mentioned [here](https://www.qgistutorials.com/en/docs/3/sampling_raster_data.html) there should be some null values but only slightly more than 100.
    - However, using `zonal_stats()` the NULL values are more than 1k. 
        - related to [this issue](https://github.com/perrygeo/python-rasterstats/issues/105)? not sure.